In [1]:
from utils.processor import *
from utils.solver import *
from config import *
from utils.tom_api import *

In [2]:
data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')
merged_datapath = os.path.join(data_dir, 'parking-meters_merged_twins.csv')
origin = (HOME['lat'], HOME['lon'])
# dest = (PACIFIC_CENTER['lat'], PACIFIC_CENTER['lon'])
ttapi = TomSearchAPI(API_KEY)
dest = ttapi.find_location('Thierry Alberni')
print(dest)
dest = dest['location']
dest = (dest['lat'], dest['lon'])
dest

{'poi_name': 'Thierry', 'address': '1059 Alberni Street, Vancouver BC V6E 1A1', 'location': {'lat': 49.28507, 'lon': -123.12318}}


(49.28507, -123.12318)

In [4]:
topK = 10
stay_duration = 1.5 * 60 
parking_budget = 8.00

processor = Processor(datapath, merged_datapath, origin, dest, topK,
                      stay_duration, parking_budget, load_merged=True)
variables = processor.form_variables_dict()
variables

(3655, 24)
(3655, 25)
IDX - 0
Time limit: 2 	 Rate: 8.0
~~~~~~~~~~~~~~~~~~~~ Single ~~~~~~~~~~~~~~~~~~~~
Skipping due to fee constraint
IDX - 1
Time limit: 2 	 Rate: 8.0
******************** Twin **********************
Skipping due to fee constraint
IDX - 2
Time limit: 2 	 Rate: 8.0
~~~~~~~~~~~~~~~~~~~~ Single ~~~~~~~~~~~~~~~~~~~~
Skipping due to fee constraint
IDX - 3
Time limit: 2 	 Rate: 4.0
~~~~~~~~~~~~~~~~~~~~ Single ~~~~~~~~~~~~~~~~~~~~
*** Travel time: 24.983333333333334 ***
out: 2 - in: 1
~~~ Parking Status: 2 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 6.0, 'savings': 2.0, 'expected_avail': 2, 'et_next_out': 12.0, 'et_next_in': 22.0, 'walk_time': 1.333, 'travel_et': 24.983333333333334}, 'meterid': '650718', 'latln': (49.28495658101305, -123.12415802048334)}
IDX - 4
Time limit: 2 	 Rate: 8.0
******************** Twin **********************
Skipping due to fee constraint
IDX - 5
Time limit: 2 	 Rate: 8.0
******************** Twin **********************
Skipping due to 

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 6.0,
   'savings': 2.0,
   'expected_avail': 2,
   'et_next_out': 12.0,
   'et_next_in': 22.0,
   'walk_time': 1.333,
   'travel_et': 24.983333333333334},
  'meterid': '650718',
  'latln': (49.28495658101305, -123.12415802048334)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 7.5,
   'savings': 0.5,
   'expected_avail': 3,
   'et_next_out': 9.0,
   'et_next_in': 24.0,
   'walk_time': 1.667,
   'travel_et': 25.166666666666668},
  'meterid': "('111103', '111101')",
  'latln': (49.28593363898792, -123.12324836757494)},
 'y_3': {'b_3': 1.0,
  'constraint_coeffs': {'parking_fee': 7.5,
   'savings': 0.5,
   'expected_avail': 3,
   'et_next_out': 12.0,
   'et_next_in': 25.0,
   'walk_time': 1.817,
   'travel_et': 25.2},
  'meterid': "('111105', '111107')",
  'latln': (49.28600717931792, -123.12336188536602)},
 'y_4': {'b_4': 1.0,
  'constraint_coeffs': {'parking_fee': 7.5,
   'savings': 0.5,
   'expected_avail': 4,
   

In [5]:
solver = Solver(variables)
solver.solve_LP()
soln = solver.get_soln_summary()
soln

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/d688efe57d9d40aea3ad40a80b7017ac-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/d688efe57d9d40aea3ad40a80b7017ac-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 35 RHS
At line 41 BOUNDS
At line 46 ENDATA
Problem MODEL has 5 rows, 4 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 4 (0) columns and 20 (0) elements
Perturbing problem by 0.001% of 1.0013245 - largest nonzero change 1.7803928e-05 ( 0.0017803928%) - largest zero change 0
0  Obj 0 Primal inf 1.1874998 (2)
3  Obj 0.96782508
Optimal - objective value 0.96781116
Optimal objective 0.96781115

,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
3,y_4,0.357296,7.5,4,24.816667,2.550,8.0,24.0
1,y_2,0.349785,7.5,3,25.166667,1.667,9.0,24.0
0,y_1,0.260730,6.0,2,24.983333,1.333,12.0,22.0
2,y_3,0.000000,7.5,3,25.200000,1.817,12.0,25.0


# Visualization

In [6]:
import folium
from folium.features import DivIcon

fmap = folium.Map(location=dest, zoom_start=16, width="100%", height="50%")

# folium.Marker(
#     location=origin, icon=folium.Icon(color="red",icon="home", prefix='fa')
# ).add_to(fmap)

folium.Marker(
    location=dest, icon=folium.Icon(color="red",icon="location-dot", prefix='fa')
).add_to(fmap)


for i, spot in enumerate(soln.loc[soln.optimal_weight > 0]['var']):
    print(spot)
    rank = i+1
    spot_loc = variables[spot]['latln']
    folium.Marker(
        location = spot_loc,
        popup = f'Rank-{rank}',
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(7,20),
            html=f'<div style="font-size: 18pt; color : black">{rank}</div>',
            )
    ).add_to(fmap)
    fmap.add_child(folium.CircleMarker(spot_loc, radius=15))

fmap

y_4
y_2
y_1
